In [1]:
# Loading Libraries

import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

import nltk
import re
from nltk.corpus import stopwords

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose



In [2]:
df = pd.read_csv('/content/drive/MyDrive/Luminar_learning_AI/NLP/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df.shape

(18285, 5)

In [8]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
# get the independent features

X = df.drop(['label'],axis=1)

# get the dependent fetures

y= df['label']

In [10]:
tf.__version__

'2.8.2'

In [11]:
# vocabulary Size
voc_size=5000

In [12]:
# One hot rep

message = X.copy()

In [13]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [14]:
message.reset_index(inplace=True)

In [15]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
ps = PorterStemmer()


corpus=[]
for i in range(0,len(message)):
  review = re.sub('[^a-zA-Z]', ' ',message['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
  review = ' '.join(review)
  corpus.append(review)

In [33]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [34]:
onehot_rep = [one_hot(word,voc_size) for word in corpus]
onehot_rep[0]

[3768, 4103, 4918, 604, 3889, 4089, 1276, 887, 4192, 354]

In [35]:
# Padding
sent_length= 20
embedded_docs = pad_sequences(onehot_rep,padding='post',maxlen=sent_length)
embedded_docs

array([[3768, 4103, 4918, ...,    0,    0,    0],
       [2350,  469,  980, ...,    0,    0,    0],
       [3235, 3758, 4943, ...,    0,    0,    0],
       ...,
       [1789, 2558, 1032, ...,    0,    0,    0],
       [3737, 3795, 1652, ...,    0,    0,    0],
       [4503,  311, 1153, ...,    0,    0,    0]], dtype=int32)

In [36]:
embedded_docs[0]

array([3768, 4103, 4918,  604, 3889, 4089, 1276,  887, 4192,  354,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [37]:
# creating model

embedding_vector_features=100
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 100)           500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(embedded_docs,y,test_size=0.33,random_state=42)

In [25]:
# final training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 9s 9ms/step - loss: 0.2905 - accuracy: 0.8631 - val_loss: 0.1949 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1350 - accuracy: 0.9471 - val_loss: 0.2160 - val_accuracy: 0.9157
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0928 - accuracy: 0.9650 - val_loss: 0.2682 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0630 - accuracy: 0.9780 - val_loss: 0.2761 - val_accuracy: 0.9094
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0429 - accuracy: 0.9847 - val_loss: 0.3582 - val_accuracy: 0.9064
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0294 - accuracy: 0.9899 - val_loss: 0.3633 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0243 - accuracy: 0.9921 - val_loss: 0.3999 - val_accuracy: 0.9016
Epoch 

In [30]:
# Accuracy check
import numpy as np

y_pred=model.predict(X_test)

y_pred=np.where(y_pred>0.6,1,0) #AUC ROC curve

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9015741507870754